## Load Env Vars

In [1]:
from dotenv import load_dotenv
import os


# Load the environment variables from the .env file
load_dotenv()


os.environ["LANGCHAIN_PROJECT"] = "Multi-Agents-Colab-Architecture"


## Create Agents

In [2]:
from langchain_openai import AzureChatOpenAI

from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START


def create_agent(llm, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                " You are a helpful AI assistant, collaborating with other assistants."
                " If you are unable to fully answer, that's OK, another assistant "
                " will help where you left off. Execute what you can to make progress."
                " Remember to focus on your given task only, do not do another assistant work ."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " \n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    return prompt | llm


## Define State Message

In [3]:
import operator
from typing import Annotated, Sequence, TypedDict, List


class AgentState(TypedDict):
    sender: str
    messages: Annotated[Sequence[BaseMessage], operator.add]
    

## Define Agent Nodes


In [4]:
import functools


deployment_name = 'gpt-4o-mini'
code_language = 'Python'


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [result],
        "sender": name,
    }


llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)


# Test Scenarios Writer and Reviewer Agents
test_scenario_writer_agent = create_agent(
    llm=llm,
    system_message="""
        You must provide a set of coherent and well defined test scenarios for another agent to write codes,
        or improve existant test scenarions following given instructions.
        You must not provide codes or other functions that are not you primary objective.
    """ 
)
ts_writer_node = functools.partial(agent_node, agent=test_scenario_writer_agent, name="ScenarioWriter")

test_scenario_reviewer_agent = create_agent(
    llm,
    system_message="""
        You must review a set of test scenarios, 
        and if the test scenarios needs enhancement you must prefix your response with REWRITE and provide instructions on how to improve. 
        You must not provide codes or other functions that are not you primary objective.
    """,
)
ts_reviewer_node = functools.partial(agent_node, agent=test_scenario_reviewer_agent, name="ScenarioReviewer")

# Test Code Writer and Reviewer
test_code_writer_agent = create_agent(
    llm=llm,
    system_message=f"You must write {code_language} code for test scenarios",
)
tc_writer_node = functools.partial(agent_node, agent=test_code_writer_agent, name="CodeWriter")

test_code_reviewer_agent = create_agent(
    llm=llm,
    system_message=f"""
        You must review {code_language} code, 
        and if the code needs enhancement you must prefix your response with REWRITE and provide instructions on how to improve. 
        You must not provide codes or other functions that are not you primary objective.
    """,
)
tc_reviewer_node = functools.partial(agent_node, agent=test_code_reviewer_agent, name="CodeReviewer")

## Define Edge Logic

In [5]:
from typing import Literal


def router(state) -> Literal["__end__", "continue", "rewrite"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "REWRITE" in last_message.content:
        return "rewrite"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    return "continue"

## Define the Graph

In [6]:
# Create Workflow
workflow = StateGraph(AgentState)

workflow.add_node("ScenarioWriter", ts_writer_node)
workflow.add_node("ScenarioReviewer", ts_reviewer_node)
workflow.add_node("CodeWriter", tc_writer_node)
workflow.add_node("CodeReviewer", tc_reviewer_node)

workflow.add_conditional_edges(
    "ScenarioWriter",
    router,
    {
        "continue": "ScenarioReviewer", 
        "__end__": END
    },
)

workflow.add_conditional_edges(
    "ScenarioReviewer",
    router,
    {
        "continue": "CodeWriter", 
        "rewrite": "ScenarioWriter", 
        "__end__": "CodeWriter"
    },
)

workflow.add_conditional_edges(
    "CodeWriter",
    router,
    {
        "continue": "CodeReviewer",
        "__end__": END
    },
)

workflow.add_conditional_edges(
    "CodeReviewer",
    router,
    {
        "continue": "ScenarioWriter", 
        "rewrite": "CodeWriter", 
        "__end__": END
    },
)


workflow.add_edge(START, "ScenarioWriter")
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display


try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [8]:
objective = """
    As a user, I want to securely log in to the application using my email and password, so I can access my personalized dashboard and features. 
    The system should also support role-based access control to restrict access to specific areas of the application based on user roles (e.g., admin, user, guest)."
"""

events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content=f" I would like for you to write 3 test scenarios, for this this objective : {objective}"
                f" for each test scenario I would like a {code_language} code script."
                " Once you code it up, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150},
)


In [ ]:
for s in events:
    print("#"*50)
    for k,v  in s.items():
        print(f"{k} : \n", s[k]['messages'][-1].content)
    print()